In [ ]:
pip install -r requirements.txt

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Dense, Dropout, LSTM
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot as plt

In [3]:
base = pd.read_csv('petr4_treinamento.csv') # Importando a base de dados
base = base.dropna() # Removendo os valores nulos
base_treinamento = base.iloc[:, 1:2].values # Pegando somente a coluna de abertura
base_valor_maximo = base.iloc[:, 2:3].values # Pegando somente a coluna de valor máximo

In [4]:
normalizador = MinMaxScaler(feature_range=(0, 1)) # Normaliza os dados
base_treinamento_normalizada = normalizador.fit_transform(base_treinamento) # Normaliza os dados de treinamento e teste com base no valor máximo e mínimo
base_valor_maximo_normalizada = normalizador.fit_transform(base_valor_maximo) # Normaliza os dados de treinamento e teste com base no valor máximo e mínimo

In [5]:
X = [] # Prepara a estrutura de dados para o treinamento
y1 = [] # Prepara a estrutura de dados para o treinamento
y2 = [] # Prepara a estrutura de dados para o treinamento
for i in range(90, 1242): # 90 registros anteriores
    X.append(base_treinamento_normalizada[i-90:i, 0]) # Pega os 90 registros anteriores
    y1.append(base_treinamento_normalizada[i, 0]) # Pega o registro atual de abertura
    y2.append(base_valor_maximo_normalizada[i, 0]) # Pega o registro atual de valor máximo
X, y1, y2 = np.array(X), np.array(y1), np.array(y2) # Transforma em array 
X = np.reshape(X, (X.shape[0], X.shape[1], 1)) # Transforma em 3D para o LSTM

In [6]:
y = np.column_stack((y1, y2)) # Junta os valores de abertura e valor máximo

In [ ]:
regressor = Sequential() # Inicializa a rede neural
regressor.add(InputLayer(input_shape=(X.shape[1], 1))) # Adiciona a camada de entrada
regressor.add(LSTM(units=100, return_sequences=True)) # Adiciona a camada LSTM com 100 neurônios e retorna a sequência de valores para a próxima camada
regressor.add(Dropout(0.3)) # Adiciona a camada de dropout para evitar overfitting

regressor.add(LSTM(units=50, return_sequences=True)) # Adiciona a camada LSTM com 50 neurônios e retorna a sequência de valores para a próxima camada
regressor.add(Dropout(0.3)) # Adiciona a camada de dropout para evitar overfitting

regressor.add(LSTM(units=50)) # Adiciona a camada LSTM com 50 neurônios
regressor.add(Dropout(0.3)) # Adiciona a camada de dropout para evitar overfitting

regressor.add(Dense(units=2, activation='linear')) # Adiciona a camada de saída com 2 neurônios

In [9]:
regressor.compile(optimizer='rmsprop', loss='mean_squared_error', metrics=['mean_absolute_error']) # Compila a rede neural

In [ ]:
regressor.fit(X, y, epochs=100, batch_size=32) # Treina a rede neural

In [11]:
base_teste = pd.read_csv('petr4_teste.csv') # Importa a base de teste
y_open = base_teste.iloc[:, 1:2].values # Pega somente a coluna de abertura
y_high = base_teste.iloc[:, 2:3].values # Pega somente a coluna de valor máximo

In [12]:
base_completa = pd.concat((base['Open'], base_teste['Open']), axis=0) # Concatena a base de treinamento com a base de teste
entradas = base_completa[len(base_completa) - len(base_teste) - 90:].values # Pega os 90 registros anteriores ao teste 
entradas = entradas.reshape(-1, 1) # Transforma em 1D
entradas = normalizador.transform(entradas) # Normaliza os dados de entrada

In [13]:
X_teste = [] # Prepara a estrutura de dados para o teste
for i in range(90, 112): # 90 registros anteriores
    X_teste.append(entradas[i-90:i, 0]) # Pega os 90 registros anteriores
X_teste = np.array(X_teste) # Transforma em array
X_teste = np.reshape(X_teste, (X_teste.shape[0], X_teste.shape[1], 1)) # Transforma em 3D para o LSTM

In [ ]:
previsoes = regressor.predict(X_teste) # Faz a previsão com base nos dados de teste normalizados
previsoes = normalizador.inverse_transform(previsoes) # Desnormaliza os dados de previsão

In [ ]:
from sklearn.metrics import mean_absolute_error
print(mean_absolute_error(y_open, previsoes[:, 0])) # Calcula o erro médio absoluto para a abertura
print(mean_absolute_error(y_high, previsoes[:, 1])) # Calcula o erro médio absoluto para o valor máximo

In [ ]:
plt.plot(y_open, color='red', label='Preço Abertura Real') # Plota o gráfico com os valores reais de abertura
plt.plot(y_high, color='black', label='Preço Máximo Real') # Plota o gráfico com os valores reais de valor máximo

plt.plot(previsoes[:, 0], color='blue', label='Preço Abertura Previsto') # Plota o gráfico com os valores previstos de abertura
plt.plot(previsoes[:, 1], color='orange', label='Preço Máximo Previsto') # Plota o gráfico com os valores previstos de valor máximo

plt.title('Previsão de preços das ações da Petrobras') # Adiciona o título ao gráfico
plt.xlabel('Tempo') # Adiciona o rótulo do eixo x
plt.ylabel('Valor') # Adiciona o rótulo do eixo y
plt.legend() # Adiciona a legenda